<h1 align="center">Permutations</h1>

<h2>Changing to correct directory</h2>

In [1]:
import os
from pathlib import Path

os.chdir(Path.cwd().parent)
assert Path.cwd().name == "src" # make sure it is the correct folder

In [2]:
import random
import pandas as pd

import tournament_simulations.data_structures as ds
import tournament_simulations.schedules.permutation as pmt
import tournament_simulations.schedules.round_robin as rr

In [3]:
from example_notebooks.matches_df_example import matches_df_permutations
matches = ds.Matches(matches_df_permutations)

matches.df

home   away winner
id date number                     
1  0                a      d      h
   0                b      c      d
   0                a      b      d
   1                c      b      a
   1                d      b      d
   2                c      a      a
   2                a      d      a
   3                b      c      h
2  0              one  three      d
   0              two    one      d
   1              two  three      a
   1              one    two      h
   2            three    two      h
   2            three    one      a

In [4]:
matches.team_names_per_id

id
1         [a, b, c, d]
2    [one, three, two]
Name: teams, dtype: object

<h2>Tournament Schedule</h2>

> Create a tournament schedule for each "id"

<h3>Same Function for all Tournaments</h3>

In [5]:
random.seed(1)

def create_single_round_robin(num_teams):
    return list(rr.SingleRoundRobin.from_num_teams(num_teams).get_full_schedule(1))

id_to_num_teams = matches.team_names_per_id.apply(len)

schedule = pmt.TournamentSchedule.from_functions(
    func_schedule=create_single_round_robin, 
    id_to_parameters=id_to_num_teams.apply(lambda cell: [cell]) # it needs to be a list of parameters
)
schedule.series.explode()

id
1    ((2, 3), (1, 0))
1    ((0, 3), (1, 2))
1    ((1, 3), (2, 0))
2           ((1, 2),)
2           ((1, 0),)
2           ((2, 0),)
Name: schedule, dtype: object

<h3>Specific Function for each Tournament</h3>

In [6]:
random.seed(1)

def create_single_round_robin(team_names, num_schedules):
    return list(rr.SingleRoundRobin.from_team_names(team_names).get_full_schedule(num_schedules))

id_to_scheduling_function = pd.Series( # Can be any Mapping entity
    index=matches.df.index.get_level_values("id").unique().tolist(),
    data=[
        lambda team_names: create_single_round_robin(team_names, 3),
        lambda team_names: create_single_round_robin(team_names, 2),
    ]
)

schedule = pmt.TournamentSchedule.from_functions(
    func_schedule=id_to_scheduling_function,
    id_to_parameters=matches.team_names_per_id.apply(lambda cell: [cell]) # it needs to be a list of parameters
)
schedule.series.explode()

id
1    ((c, d), (b, a))
1    ((a, d), (b, c))
1    ((b, d), (c, a))
1    ((b, d), (c, a))
1    ((a, d), (b, c))
1    ((c, d), (b, a))
1    ((c, d), (b, a))
1    ((a, d), (b, c))
1    ((b, d), (c, a))
2     ((two, three),)
2     ((one, three),)
2       ((two, one),)
2       ((two, one),)
2     ((one, three),)
2     ((two, three),)
Name: schedule, dtype: object

<h2>Match Dates Numbers</h2>

> Dates each pair (A, B) played in.

> It is padded with -1 so that all entries for a tournament id have the same length.

In [7]:
match_date_numbers = pmt.MatchDateNumbers.from_matches(matches)
match_date_numbers.series

id  home   away 
1   a      b        [0, -1]
           d         [0, 2]
    b      c         [0, 3]
    c      a        [2, -1]
           b        [1, -1]
    d      b        [1, -1]
2   one    three        [0]
           two          [1]
    three  one          [2]
           two          [2]
    two    one          [0]
           three        [1]
Name: date number, dtype: object

<h2>Permutation</h2>

> Permute matches in all tournaments

In [8]:
matches_to_permute = pmt.PermuteMatches(matches)

<h3>Manual</h3>

In [9]:
random.seed(1)

index_manual = pmt.OrderedIndex(
    pd.Series(
        index=["1", "2"],
        data=[
            [
                # id, date number, home, away
                ("1", 1,     "c",     "b"),
                ("1", 0,     "a",     "d"),
                ("1", 2,     "a",     "d"),
                ("1", 2,     "c",     "a"),
                ("1", 3,     "b",     "c"),
                ("1", 0,     "b",     "c"),
                ("1", 0,     "a",     "b"),
                ("1", 1,     "d",     "b"),
            ],
            [
                ("2", 1,   "one",   "two"),
                ("2", 1,   "two", "three"),
                ("2", 0,   "one", "three"),
                ("2", 2, "three",   "one"),
                ("2", 0,   "two",   "one"),
                ("2", 2, "three",   "two"),
            ]
        ]
    )
)

In [10]:
permuted_manual = matches_to_permute.permute_matches(index_manual)

custom_date_numbers = matches.df.groupby("id", observed=True).size().apply(range).explode().to_list()
permuted_custom_date_numbers = matches_to_permute.permute_matches(index_manual, custom_date_numbers)

pd.concat(
    [
        matches.df.reset_index("date number"),
        permuted_manual.df.reset_index("date number"),
        permuted_custom_date_numbers.df.reset_index("date number"),
    ], 
    axis="columns",
    keys=["original", "permuted", "permuted custom date numbers"],
)

original                         permuted                       \
   date number   home   away winner date number   home   away winner   
id                                                                     
1            0      a      d      h           0      c      b      a   
1            0      b      c      d           0      a      d      h   
1            0      a      b      d           0      a      d      a   
1            1      c      b      a           1      c      a      a   
1            1      d      b      d           1      b      c      h   
1            2      c      a      a           2      b      c      d   
1            2      a      d      a           2      a      b      d   
1            3      b      c      h           3      d      b      d   
2            0    one  three      d           0    one    two      h   
2            0    two    one      d           0    two  three      a   
2            1    two  three      a           1    one  three      d   
2            1    one    two      h           1  three    one      a   
2            2  three    two      h           2    two    one      d   
2            2  three    one      a           2  three    two      h   

   permuted custom date numbers                       
                    date number   home   away winner  
id                                                    
1                             0      c      b      a  
1                             1      a      d      h  
1                             2      a      d      a  
1                             3      c      a      a  
1                             4      b      c      h  
1                             5      b      c      d  
1                             6      a      b      d  
1                             7      d      b      d  
2                             0    one    two      h  
2                             1    two  three      a  
2                             2    one  three      d  
2                             3  three    one      a  
2                             4    two    one      d  
2                             5  three    two      h

<h3>Random Index</h3>

In [11]:
random.seed(1)

def create_double_rr(team_names, num_schedules):
    return list(rr.DoubleRoundRobin.from_team_names(team_names).get_full_schedule(num_schedules))

num_schedule_per_id = matches.home_vs_away_count_per_id.groupby("id", observed=True).max()
id_to_parameters = pd.concat([matches.team_names_per_id, num_schedule_per_id], axis="columns")

tournament_schedule = pmt.TournamentSchedule.from_functions(
    func_schedule=create_double_rr,
    id_to_parameters=id_to_parameters.agg(tuple, axis="columns"),
)

random_index = pmt.OrderedIndex.from_schedule__date_numbers(
    tournament_schedule,
    match_date_numbers.create_shuffled_copy()
)

In [12]:
permuted_manual = matches_to_permute.permute_matches(random_index)

pd.concat(
    [matches.df, permuted_manual.df], 
    axis="columns",
    keys=["original", "permuted"],
)

original               permuted              
                   home   away winner     home   away winner
id date number                                              
1  0                  a      d      h        a      d      h
   0                  b      c      d        b      c      d
   0                  a      b      d        c      b      a
   1                  c      b      a        d      b      d
   1                  d      b      d        a      d      a
   2                  c      a      a        b      c      h
   2                  a      d      a        c      a      a
   3                  b      c      h        a      b      d
2  0                one  three      d    three    one      a
   0                two    one      d      two  three      a
   1                two  three      a      one    two      h
   1                one    two      h      two    one      d
   2              three    two      h    three    two      h
   2              three    one      a      one  three      d